In [34]:
import pandas as pd
import numpy as np
import re
import mysql.connector
import csv
import os
import wget
import xml.dom.minidom
from pprint import pprint
import pickle
import nltk
from tqdm import tqdm
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [4]:
db = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="root",
  database="opencopora.ru"
)
sql = db.cursor()

In [5]:
# список отобранных id источников (исключены битые статьи и служебные разделы, а также некорректно размеченные источники)
source_ids = [
    1,
    8,
    56,
    184,
    226,
    806,
    1651,
    1675,
    1724,
    2037,
    3469,
    3477,
    3984,
    3994
]
# получаем документы
sql.execute("SELECT DISTINCT bk.book_id, bk.book_name, bk.parent_id, bk.syntax_on, src.url, urls.`filename` \
FROM `opencopora.ru`.books bk \
LEFT JOIN `opencopora.ru`.`sources` src \
ON src.book_id=bk.book_id \
LEFT JOIN `opencopora.ru`.`downloaded_urls` urls \
ON src.url=urls.url")
 
document_sources = pd.DataFrame.from_records(sql.fetchall(), columns=['book_id', 'book_name', 'parent_id', 'syntax_on', 'url', 'filename'])
document_sources.set_index('book_id')
document_sources.head(10)

,book_id,book_name,parent_id,syntax_on,url,filename
0,1,"""Частный корреспондент""",0,0,http://chaskor.ru,4e293b41a67923.57187979
1,2,00021 Школа злословия,1,0,http://www.chaskor.ru/article/shkola_zlosloviy...,4dda53c1719c23.72408057
2,3,00022 Последнее восстание в Сеуле,1,0,http://www.chaskor.ru/article/poslednee_vossta...,4de630ae805107.92951684
3,4,00023 За кота - ответишь!,1,0,http://www.chaskor.ru/article/za_kota_-_otveti...,4de630b496c6b1.81320751
4,5,00024 Быстротечный кинороман,1,0,http://www.chaskor.ru/article/bystrotechnyj_ki...,4de630bae523e8.68387316
5,6,00014 Холодная ванна возвращает силы,1,0,http://www.chaskor.ru/article/holodnaya_vanna_...,4dda535864d0f3.68074327
6,7,00031 Рецессия в Латвии и Эстонии,1,0,http://www.chaskor.ru/article/retsessiya_v_lat...,4de630c64a52d1.08149203
7,8,Википедия,0,0,http://ru.wikipedia.org,None
8,9,06037 100 дней Обамы: iТоги,1,0,http://www.chaskor.ru/article/100_dnej_obamy_i...,4de6619227c894.71103195
9,10,01961 100 миллиардов может и не хватить,1,0,http://www.chaskor.ru/article/100_milliardov_m...,4de630d4bc7c33.59812927


In [23]:
# извлечение всех предложений из документа
def extract_sentences(doc_id):
    # получаем из базы все предложения из документа, запоминая разбиение на абзацы
    sql.execute("SELECT sent.sent_id, sent.par_id, sent.`source`, par.book_id, par.pos as par_pos, sent.pos as sent_pos \
    FROM `opencopora.ru`.sentences sent \
    RIGHT JOIN `opencopora.ru`.paragraphs par \
    ON sent.par_id=par.par_id \
    WHERE par.book_id=%s ORDER BY par_pos, sent_pos", (doc_id,))
    return pd.DataFrame.from_records(sql.fetchall(), columns=['sent_id', 'par_id', 'source', 'book_id', 'par_pos', 'sent_pos'])

# создание текста из предложений документа
def create_text(sentences):
    text = ''
    for index, row in sentences.iterrows():
        text += row['source'] + ' '
    return text

# получить путь к папке с файлами одного документа
def get_data_dir(doc_id):
    doc_id    = str(doc_id)
    directory = 'dataset/' + doc_id
    if not os.path.exists(directory):
        os.makedirs(directory)
    return directory

# получить url сохраненной копии документа
def get_saved_source_url(filename):
    return 'http://opencorpora.org/files/saved/' + str(filename) + '.html'

# сохранение текста, разбитого на предложения вручную
def save_sentences(doc_id):
    doc_id    = str(doc_id)
    directory = get_data_dir(doc_id)
    path      = directory + '/' + doc_id + '-sentences.csv'
    sentences = extract_sentences(doc_id)
    sentences.to_csv(path, index=False, escapechar='\\', quoting=csv.QUOTE_NONNUMERIC)
    
# сохранение текста, созданного из предложений
def save_text(doc_id):
    doc_id    = str(doc_id)
    directory = get_data_dir(doc_id)
    path      = directory + '/' + str(doc_id) + '-text.txt'
    text      = create_text(extract_sentences(doc_id))
    with open(path, 'w') as txt_file:
        txt_file.write(text)

# сохранение оригинала документа
def save_original_document(filename, doc_id):
    doc_id       = str(doc_id)
    directory    = get_data_dir(doc_id)
    path         = directory + '/' + doc_id + '-original.html'
    document_url = get_saved_source_url(filename)
    wget.download(document_url, path)
    
# сохранение данных полной ручной разметки документа в xml
def save_annotations(doc_id):
    doc_id    = str(doc_id)
    src_path  = 'opcorpora-documents/' + doc_id + '.xml'
    dest_path = get_data_dir(doc_id) + '/' + doc_id + '-annotations.xml'
    # форматируем xml чтобы его было легче читать
    raw_xml = xml.dom.minidom.parse(src_path)
    with open(dest_path, 'w') as xml_file:
        xml_file.write(raw_xml.toprettyxml())
        
# автоматическое разбиение текста на предложения - точка отсчета для сравнения целевых показателей будущей модели
# разбивает текст документа на предложения, пользуясь стандартной моделью из NLTK
def save_baseline(doc_id):
    doc_id = str(doc_id)
    # проверяем что папка не служебная
    if doc_id.startswith('.'):
        return
    # папка должна существовать
    if not os.path.exists(doc_id):
        return FileNotFoundError('ERR: не найдена папка для документа id=' + doc_id)
    
    text_path = 'dataset/' + doc_id + '/' + doc_id + '-text.txt'
    text_file = open(text_path)
    text      = text_file.read()
    sentences = nltk.sent_tokenize(text)
    # производим автоматическое разбиение на предложения для получения baseline
    sentences_path = 'dataset/' + doc_id + '/' + doc_id + '-sentences-auto.csv'
    df = pd.DataFrame(data={"source": sentences})
    df.to_csv(sentences_path, sep=',', index=True, quoting=csv.QUOTE_NONNUMERIC, index_label="sent_pos")
        
# получение списка документов для одного источника
def get_source_documents(source_id):
    sql.execute("SELECT DISTINCT bk.book_id, bk.book_name, bk.parent_id, bk.syntax_on, src.url, urls.`filename` \
    FROM `opencopora.ru`.books bk \
    RIGHT JOIN `opencopora.ru`.`sources` src \
    ON src.book_id=bk.book_id \
    RIGHT JOIN `opencopora.ru`.`downloaded_urls` urls \
    ON src.url=urls.url \
    WHERE bk.parent_id=%s", (source_id,))
    return pd.DataFrame.from_records(sql.fetchall(), columns=['book_id', 'book_name', 'parent_id', 'syntax_on', 'url', 'filename'])

# получение данных одного документа по id
def get_document_by_id(doc_id):
    sql.execute("SELECT DISTINCT bk.book_id, bk.book_name, bk.parent_id, bk.syntax_on, src.url, urls.`filename` \
    FROM `opencopora.ru`.books bk \
    RIGHT JOIN `opencopora.ru`.`sources` src \
    ON src.book_id=bk.book_id \
    RIGHT JOIN `opencopora.ru`.`downloaded_urls` urls \
    ON src.url=urls.url \
    WHERE bk.book_id=%s", (doc_id,))
    return pd.DataFrame.from_records(sql.fetchall(), columns=['book_id', 'book_name', 'parent_id', 'syntax_on', 'url', 'filename'])

# оценить качество разбиения текста на предложения (от 0 до 100 %)
def calculate_split_score(sentence_count, hit_count):
    if hit_count == 0:
        return 0
    return round((hit_count / sentence_count) * 100, 2)
    
# подсчитать статистику точности разбиения на предложения по одному документу
def calculate_document_stats(doc_id):
    manual_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences.csv'
    manual_df = pd.read_csv(manual_sentences_file)
    
    auto_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences-auto.csv'
    auto_df = pd.read_csv(auto_sentences_file)
    
    # получаем списки предложений из обучающей и тестовой выборки
    manual_sentences = list(manual_df['source'].values)
    auto_sentences   = list(auto_df['source'].values)
    
    # всего совпадений (одинаково разбитых предложений)
    total_hits           = set(manual_sentences).intersection(set(auto_sentences))
    # определяем сколько (и какие) предложения из ручного разбиения удалось автоматически повторить
    manual_split_hits    = set(manual_sentences).intersection(set(auto_sentences))
    manual_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    
    # определяем сколько (и какие) предложения из автоматического разбиения совпадают с ручным
    auto_split_hits    = set(auto_sentences).intersection(set(manual_sentences))
    auto_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    
    return {
        "doc_id" : doc_id,
        # статистика разбиения
        "stats" : {
            # общее количество предложений которые одинаково разбиты вручную и автоматически
            "total_hit_count"          : len(total_hits),
            # результаты ручного разбиения
            "manual_split_count"       : len(manual_sentences),
            "manual_split_misses_count": len(manual_split_misses),
            # сколько процентов ручного разбиения удалось повторить автоматически
            "manual_split_score"       : calculate_split_score(len(manual_sentences), len(manual_split_hits)),
            # результаты авторазбиения
            "auto_split_count"         : len(auto_sentences),
            "auto_split_misses_count"  : len(auto_split_misses),
            # сколько процентов автоматического разбиения соответствует ручной проверке
            "auto_split_score"         : calculate_split_score(len(auto_sentences), len(auto_split_hits)),
        },
        # результаты разбиения (текст предложений по группам)
        "results" : {
            "manual_sentences"   : manual_sentences,
            "auto_sentences"     : auto_sentences,
            # все предложения, одинаково разбитые и вручную и автоматически
            "total_hits"         : total_hits,
            # количество предложений для которых не удалось построить автоматическое разбиение
            "manual_split_misses": manual_split_misses,
            # количество предложений в автоматическом разбиении которые были выделены неправильно
            "auto_split_misses"  : auto_split_misses,
        } 
    }

# вычислить точность изначального разбиения (без обучения токенизатора)
def calculate_baseline_document_score(doc_id):
    data = calculate_document_stats(doc_id)
    return data["stats"]["auto_split_score"]

# вычислить точность разбиения обученной модели
def calculate_trained_document_score(doc_id, tokenizer):
    manual_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences.csv'
    manual_df = pd.read_csv(manual_sentences_file)
    
    trained_sentences_file = 'dataset/' + doc_id + '/' + doc_id + '-sentences-trained.csv'
    trained_df = pd.read_csv(trained_sentences_file)
    
    # получаем списки предложений из обучающей и тестовой выборки
    manual_sentences = list(manual_df['source'].values)
    trained_sentences = list(trained_df['source'].values)
    
    # всего совпадений (одинаково разбитых предложений)
    total_hits           = set(manual_sentences).intersection(set(trained_sentences))
    # определяем сколько (и какие) предложения из ручного разбиения удалось автоматически повторить
    manual_split_hits    = set(manual_sentences).intersection(set(trained_sentences))
    manual_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    
    # определяем сколько (и какие) предложения из автоматического разбиения совпадают с ручным
    trained_split_hits    = set(trained_sentences).intersection(set(manual_sentences))
    trained_split_misses  = set(manual_sentences).difference(set(manual_split_hits))
    
    trained_result = {
        "doc_id" : doc_id,
        "stats" : {
            "total_hit_count"          : len(total_hits),
            "manual_split_count"       : len(manual_sentences),
            "manual_split_misses_count": len(manual_split_misses),
            "manual_split_score"       : calculate_split_score(len(manual_sentences), len(manual_split_hits)),
            "trained_split_count"         : len(trained_sentences),
            "trained_split_misses_count"  : len(trained_split_misses),
            "trained_split_score"         : calculate_split_score(len(trained_sentences), len(trained_split_hits)),
        },
        "results" : {
            "manual_sentences"      : manual_sentences,
            "trained_sentences"     : trained_sentences,
            "total_hits"            : total_hits,
            "manual_split_misses"   : manual_split_misses,
            "trained_split_misses"  : trained_split_misses,
        } 
    }
    return trained_result

In [33]:
# подготовить все данные и признаки для одного документа корпуса
def prepare_document(doc_id):
    doc_id = str(doc_id)
    doc    = get_document_by_id(doc_id)
    # получаем из документа все предложения на которые он был разбит вручную
    # собираем предложения обратно в текст на котором будем обучать модель для автоматического разбиения
    try:
        save_sentences(doc_id)
        save_text(doc_id)
    except Exception:
        print("Не удалось произвести разбор предложений документа" + str(doc_id))
    # сохраняем разметку документа
    try:
        save_annotations(doc_id)
    except Exception:
        print("Не удалось сохранить аннотацию документа " + str(doc_id))
    # сохраняем оригинал документа
    try:
        save_original_document(doc['filename'], doc_id)
    except Exception:
        print("Не удалось сохранить оригинал документа " + str(doc_id))

# создать csv-файл со списком всех документов корпуса, добавить дополнительные признаки для каждого документа
def prepare_document_index():
    dataset_folders = os.listdir('dataset')
    document_scores = []
    baseline_scores = []
    for folder in tqdm(dataset_folders):
        if folder.startswith('.'):
            continue
        doc_score = calculate_baseline_document_score(folder)
        document_scores.append({
            "doc_id"         : folder,
            "baseline_score" : doc_score
        })
        baseline_scores.append(doc_score)
    df = pd.DataFrame(document_scores)
    df.to_csv("documents.csv", sep=',', index=False, quoting=csv.QUOTE_NONNUMERIC)
    print("Mean baseline document score:")
    print(np.mean(baseline_scores))
    print("Median baseline document score:")
    print(np.median(baseline_scores))
    
# подготовить обучающую выборку для модели разбора текста на предложения
# (Основная функция на этом шаге)
def prepare_dataset():
    for index, doc in tqdm(document_sources.iterrows()):
        # не создаем заново те документы для которых уже извлечены данные
        if os.path.exists('dataset/' + str(doc['book_id'])):
            continue
        # сохраняем предложения и текст
        try:
            save_sentences(doc['book_id'])
            save_text(doc['book_id'])
        except Exception:
            print("Не удалось произвести разбор предложений документа" + str(doc['book_id']))
            continue
        # сохраняем разметку документа
        try:
            save_annotations(doc['book_id'])
        except Exception:
            print("Не удалось сохранить аннотацию документа " + str(doc['book_id']))
        # сохраняем оригинал документа
        try:
            save_original_document(doc['filename'], doc['book_id'])
        except Exception:
            print("Не удалось сохранить оригинал документа " + str(doc['book_id']))

# @see https://nlpforhackers.io/splitting-text-into-sentences/
# обучение nltk-токенизатора для работы с предложениями этого языкового корпуса
def train_nltk_tokenizer():
    text    = ""
    # считыввем все предложения корпуа, по 1 на каждой строке
    file    = open("train.txt")
    lines   = [line.rstrip('\n') for line in file]
    for line in lines:
        text += line
    # создаем модель для разбиения текста
    trainer = PunktTrainer()
    trainer.INCLUDE_ALL_COLLOCS = True
    trainer.train(text)
    # создаем модель, обученную на текстах корпуса
    tokenizer = PunktSentenceTokenizer(trainer.get_params())

    # Test the tokenizer on a piece of text
    #sentences = open('dataset/10/10-text.txt').read()
    #print(tokenizer.tokenize(sentences))
    # View the learned abbreviations
    #print(tokenizer._params.abbrev_types)
    # set([...])

    # Here's how to debug every split decision
    #for decision in tokenizer.debug_decisions(sentences):
    #    pprint(decision)
    #    print('=' * 30)
        
    return tokenizer

# сохранить разбиение созданное обученной моделью
def save_trained(doc_id, tokenizer):
    doc_id = str(doc_id)
    # проверяем что папка не служебная
    if doc_id.startswith('.'):
        return
    # папка должна существовать
    if not os.path.exists('dataset/' + doc_id):
        return FileNotFoundError('ERR: не найдена папка для документа id=' + doc_id)
    
    text_path = 'dataset/' + doc_id + '/' + doc_id + '-text.txt'
    text_file = open(text_path)
    text      = text_file.read()
    sentences = tokenizer.tokenize(text)
    # производим автоматическое разбиение на предложения для получения baseline
    sentences_path = 'dataset/' + doc_id + '/' + doc_id + '-sentences-trained.csv'
    df = pd.DataFrame(data={"source": sentences})
    df.to_csv(sentences_path, sep=',', index=True, quoting=csv.QUOTE_NONNUMERIC, index_label="sent_pos")
    
    return df
   
# вычислить и сохранить результат работы модели, обученной на текстах языкового корпуса
def save_trained_model_score(tokenizer):
    dataset_folders = os.listdir('dataset')
    document_scores = []
    trained_scores  = []
    for folder in tqdm(dataset_folders):
        if folder.startswith('.'):
            continue
        save_trained(folder, tokenizer)
        trained_score = calculate_trained_document_score(folder, tokenizer)
        document_scores.append({
            "doc_id"        : folder,
            "trained_score" : trained_score['stats']['trained_split_score']
        })
        trained_scores.append(trained_score['stats']['trained_split_score'])
        
    df = pd.DataFrame(document_scores)
    df.to_csv("documents-trained.csv", sep=',', index=False, quoting=csv.QUOTE_NONNUMERIC)
    print("Mean trained model score:")
    print(np.mean(trained_scores))
    print("Median trained model score:")
    print(np.median(trained_scores))
    
# сохранить модель на диск для публикации
def save_trained_model_to_file(tokenizer):
    f = open('sentence-tokenizer-model.pickle', 'wb')
    pickle.dump(tokenizer, f)
    f.close()
    #tokenizer = nltk.data.load('nltk:tokenizers/punkt/english.pickle')

In [32]:
prepare_document_index()
# обучаем токенизатор на текстах корпуса
new_tokenizer = train_nltk_tokenizer()
save_trained_model_score(new_tokenizer)
save_trained_model_to_file(new_tokenizer)

Mean baseline document score:
83.95318390219053
Median baseline document score:
87.5


100%|██████████| 108960/108960 [00:00<00:00, 440516.66it/s]


Mean trained model score:
85.04055017829855
Median trained model score:
88.24


In [19]:
# среднее значение для качества baseline-разбиения

83.95318390219053

In [26]:
# calculate_document_stats("2")

In [ ]:
# создать обучающую выборку для автоматического разбиения текста на предложения
# создает текстовый файл в который записаны все предложения из всех текстов языкового корпуса
# (каждое предложение с новой строки)
def create_sentence_train_set():
    sql.execute("SELECT sent_id, source FROM sentences")
    # получаем все размеченные вручную предложения из всех текстов корпуса
    train_sentences = pd.DataFrame.from_records(sql.fetchall(), columns=['sent_id', 'source'])
    train_text      = ''
    
    for index, row in train_sentences.iterrows():
        train_text += row['source'] + "\n"
        
    with open('sentence-train.txt', 'w') as txt_file:
        txt_file.write(train_text)